In [3]:
import operator

def preprocess(sentence): #전처리 하는 함수
    preprocessed_sentence = sentence.strip().split(" ") # 주어진 문장에서 양쪽 공백을 제거하고 공백을 기준으로 문장을 분할
    return preprocessed_sentence  # 분할된 토큰은 문자열 리스트로 반환

def indexing(file_name): #기존 파일을 토큰으로 분활후 리스트에 저장하는 함수
    file_tokens_pairs = []#파일 토큰 짝 리스트 생성
    lines = open(file_name, "r", encoding="utf8").readlines()
    for line in lines: #리스트 형태로 추출한 파일의 텍스트를 line에 하나씩 넣기
        tokens = preprocess(line) #양쪽 공백없이 토큰에 입력하고 토큰을 단어 별로 쪼개기
        file_tokens_pairs.append(tokens) #토큰을 파일 토큰 짝 리스트에 추가
    return file_tokens_pairs

def calc_similarity(preprocessed_query, preprocessed_sentences): #유사도를 계산하는 함수
    score_dict={} #유사도를 저장하기 위한 변수
    for i in range(len(preprocessed_sentences)):        
        # 시작: 대소문자 구분 없는 토큰 셋을 만들기 위한 코드
        sentence = preprocessed_sentences[i] #문장을 가져오기 
        query_str = ' '.join(preprocessed_query).lower() #전처리 된 쿼리를 소문자로 변환하고 문자열로 결합
        sentence_str = ' '.join(sentence).lower() #소문자로 변환하고 문자열로 결합 
        preprocessed_query = set(preprocess(query_str)) #소문자로 변환한 쿼리를 전처리 하고 토큰 화하기
        preprocessed_sentence = preprocess(sentence_str)#소문자로 변환된 문장으로 전처리하기
        # 끝: 대소문자 구분 없는 토큰 셋을 만들기 위한 코드               
        
        file_token_set = set(preprocessed_sentence) #토큰 집합 만들기
        all_tokens = preprocessed_query | file_token_set #쿼리와 문장의 토큰 합치기
        same_tokens = preprocessed_query & file_token_set #쿼리와 문장의 토큰 교집합 만들기
        similarity = len(same_tokens) / len(all_tokens)# 유사성 계산하기
        score_dict[i] = similarity #유사 점수를 변수에 저장
    return score_dict #유사 점수가 있는 사전 반환

In [7]:
import operator

# 1. Indexing
## https://github.com/jungyeul/korean-parallel-corpora
file_name = "jhe-koen-dev.en.txt" #파일 지정
file_tokens_pairs=indexing(file_name)

# 2. Input the query
query = input("영어 쿼리를 입력하세요.").lower() #사용자가 문자 입력
preprocessed_query = query.strip().split(" ") #전처리 쿼리에 입력한 문자 공백없이 입력
query_token_set = set(preprocessed_query) #입력받은 토큰 세트에 집합 만들기

# 3. Calculate similarities based on a same token set
score_dict = calc_similarity(query_token_set, file_tokens_pairs)#입력받은 문자열과 검색 대상 내 문장들 유사도 계산

# 4. Sort the similarity list
sorted_score_list = sorted(score_dict.items(), key = operator.itemgetter(1), reverse=True) #큰 순으로 정렬이 된 리스트 생성

# 5. Print the result
if sorted_score_list[0][1] == 0.0: # 만약 가장 유사한 문장의 점수가 0.0이면,
    print("There is no similar sentence.") #유사한 문장이 없다고 출력
else: #그게 아니라면
    print("rank", "Index", "score", "sentence", sep = "\t") # \t간격으로 출력
    rank = 1 #랭크는 1로 초기화
    for i, score  in sorted_score_list: #i와 스코어에 튜플 형태의 아이템 가져오기
        print(rank, i, score, ' '.join(file_tokens_pairs[i]), sep = "\t") #순위 인덱스 점수 문장 출력하기
        if rank == 10: # 상위 10개만 출력후
            break # 탈출
        rank = rank + 1 #순위를 증가 시키기

영어 쿼리를 입력하세요.Hello my name is jun!
rank	Index	score	sentence
1	679	0.5	My name is Mike.
2	526	0.2857142857142857	Bob is my brother.
3	538	0.2857142857142857	My hobby is traveling.
4	453	0.25	My mother is sketching them.
5	241	0.2222222222222222	My father is running with So-ra.
6	336	0.2222222222222222	My family is at the park.
7	212	0.2	My sister Betty is waiting for me.
8	505	0.18181818181818182	My little sister Annie is five years old.
9	610	0.15384615384615385	I would raise my voice and yell, "LUNCH IS READY!"
10	190	0.14285714285714285	It is Sunday.
